In [89]:
from kneed import DataGenerator, KneeLocator
import sys
import os
sys.path.append(os.path.abspath('../extraction'))
import matplotlib.pyplot as plt
import pandas as pd
import csv
import scipy.io
import json
import numpy as np
from format_data import *
from utils import *
import pickle
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from collections import Counter
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.neighbors import KNeighborsClassifier  # Importer KNeighborsClassifier
#from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import seaborn as sns
import resync as rs
from scipy.signal import find_peaks

fs = 160

In [90]:
t_pre = 0.2#0.2
t_post = 0.50#0.300
bin_width = 0.005
# Créer les bins de temps"
psth_bins = np.arange(-t_pre, t_post, bin_width)
#gc = np.arange(0, 32)

In [91]:
path = 'Z:/eTheremin/ALTAI/ALTAI_20240725_SESSION_00/'

In [92]:
file_path = path + 'headstage_0/tt.pkl'

# Charger le fichier .pkl
with open(file_path, 'rb') as file:
    tt = pickle.load(file)

# Afficher l'objet chargé
print(tt)

{'tones': array([1490., 1287., 1490., ..., 1490., 1727.,  533.]), 'triggers': [165123, 165483, 166443, 166643, 190443, 203963, 341604, 345484, 351324, 354724, 357124, 357484, 357724, 378484, 378684, 378924, 391284, 445084, 483924, 529444, 532644, 540324, 548604, 551764, 552924, 557244, 560764, 561924, 562284, 562444, 563244, 563764, 563964, 564564, 587964, 592804, 597324, 602604, 605604, 619284, 620404, 623404, 639124, 656764, 657724, 657924, 710004, 730804, 740244, 742084, 806004, 811804, 812044, 812404, 812604, 876924, 881404, 881604, 882124, 886284, 886444, 886644, 889644, 900724, 901644, 922444, 922804, 929604, 952284, 984564, 988444, 991644, 995764, 996124, 996324, 996564, 998244, 1001764, 1016244, 1021644, 1025964, 1058044, 1069444, 1069644, 1069884, 1070764, 1076764, 1085604, 1086124, 1088764, 1097244, 1105804, 1119564, 1199044, 1199404, 1199604, 1200924, 1206564, 1208604, 1257124, 1257924, 1258284, 1260124, 1271404, 1282444, 1301964, 1305124, 1306884, 1324444, 1324644, 1324804,

In [93]:
def clean_positions(positions):
    """
    Fonction de nettoyage des positions enregistrées au cours de l'expérience. Pour rappel, une valeur de -1, indique que le sujet n'a pas été détecté par le réseau de neurones.
    :param positions: 
    :return: 
    """
    y = np.where(positions == -1)[0]
    diff_y = np.diff(y)
    diff_y = np.vstack((np.arange(1, len(diff_y) + 1), diff_y)).T
    k = 0
    begin = 0
    for i, elt in diff_y:
        if elt != 1:
            positions[y[i - 1]] = positions[y[i - 1] + 1]
            positions[y[i]] = positions[y[i] - 1]
            if k != 0:
                end = y[i - 1]
                filler = np.full(shape=k, fill_value=positions[begin - 1])
                positions[begin:end] = filler
                k = 0
        else:
            if k == 0:
                begin = y[i - 1]
            k += 1
    remainder = np.where(positions == -1)[0]
    if len(remainder) != 0 and k != 0:
        pass

    return positions

In [94]:
positions = rs.load_positions_file(path)
print(positions)
positions = [file for file in positions if file != ""]
print(positions)

['Z:/eTheremin/ALTAI/ALTAI_20240725_SESSION_00/positions\\positions_tail_00_ALTAI_SESSION_00_20240725.bin', '', '', '', 'Z:/eTheremin/ALTAI/ALTAI_20240725_SESSION_00/positions\\positions_tracking_01_ALTAI_SESSION_00_20240725.bin', 'Z:/eTheremin/ALTAI/ALTAI_20240725_SESSION_00/positions\\positions_playback_01_ALTAI_SESSION_00_20240725.bin', 'Z:/eTheremin/ALTAI/ALTAI_20240725_SESSION_00/positions\\positions_tracking_02_ALTAI_SESSION_00_20240725.bin', 'Z:/eTheremin/ALTAI/ALTAI_20240725_SESSION_00/positions\\positions_playback_02_ALTAI_SESSION_00_20240725.bin', 'Z:/eTheremin/ALTAI/ALTAI_20240725_SESSION_00/positions\\positions_tracking_03_ALTAI_SESSION_00_20240725.bin', 'Z:/eTheremin/ALTAI/ALTAI_20240725_SESSION_00/positions\\positions_playback_03_ALTAI_SESSION_00_20240725.bin', 'Z:/eTheremin/ALTAI/ALTAI_20240725_SESSION_00/positions\\positions_tracking_04_ALTAI_SESSION_00_20240725.bin', 'Z:/eTheremin/ALTAI/ALTAI_20240725_SESSION_00/positions\\positions_playback_04_ALTAI_SESSION_00_2024072

In [66]:
print(len(positions))

11


In [96]:
x_nested = []
y_nested = []

for k in range(len(positions)):
    p = positions[k]
    # Attention, positions en deux dimensions !
    p = np.fromfile(p, dtype=np.int32)
    x_p = p[np.arange(0, len(p), step=2)]
    y_p = p[np.arange(1, len(p), step=2)]
    mapping = rs.Mapping(1920, 33, 2000., 7)
    clean_x_p = clean_positions(x_p)  # Pour retirer les -1.
    clean_y_p = clean_positions(y_p)
    print(len(clean_x_p))
    x_nested.append(clean_x_p)
    y_nested.append(clean_y_p)
#print(len(x_nested[0]))
x = [item for sublist in x_nested for item in sublist]
y = [item for sublist in y_nested for item in sublist]
v_x = [(x[k+1]-x[k])*160 for k in range(len(x)-1)]
v_y = [(y[k+1]-y[k])*160 for k in range(len(y)-1)]


47998
48000
48001
47997
47998
47998
47999
48000
48001
47998
47999


In [101]:
print(v_x[0:300])

[0, -480, 0, 0, 0, 0, 320, -480, 0, 0, -320, 0, -160, 0, 0, 0, -320, 320, 0, 0, 0, 0, -480, -320, 320, 0, -320, 0, 0, 320, 0, -320, 0, -160, 0, 0, 0, 0, 480, -480, 0, 0, 160, -160, 160, 0, 0, -160, 0, -320, 0, 0, 320, 0, -320, -160, 0, 0, -320, 0, 0, 320, -320, -160, -320, 960, -960, -160, 640, 0, -160, 0, -320, 0, -160, 0, 0, -320, -160, 0, 0, -320, 0, -160, 0, 0, 0, -800, 320, 0, 0, -320, 320, -320, 320, -320, 320, -320, 0, -160, -480, -320, 0, 0, 0, 0, 0, 0, 320, 0, 480, 0, 0, -320, -160, -320, 0, -160, -320, 0, -480, -160, 160, -480, 0, 320, -480, -320, -160, 160, -640, -320, -480, 0, -480, 0, -480, -480, 0, 320, -320, -480, -480, 0, -160, 0, -480, 640, 0, 480, -160, 160, 320, 160, 320, -320, 0, 320, -320, -160, -320, 0, 0, -480, 0, -160, -320, 0, 320, 0, -320, 320, -320, -160, -320, 0, -480, 320, -480, -320, 0, -480, -160, -320, 0, -160, 160, 320, -320, -480, -480, -480, -160, 160, 320, 160, 0, 0, -160, -480, -800, -480, -640, -320, -160, -480, -960, -320, -480, -640, -320, -480, 

In [97]:
print(len(x),len(y))

527989 527989


In [7]:
data = np.load(path+'headstage_0/data_0.005.npy', allow_pickle=True)
features = np.load(path+'headstage_0/features_0.005.npy', allow_pickle=True)
gc = np.load(path+'headstage_0/good_clusters.npy', allow_pickle=True)

In [22]:
print(len(clean_x_p),len(clean_y_p),len(features))
print(features[5080:5090])

47998 47998 721980
[{'Played_frequency': 1727.0, 'Condition': -1.0, 'Block': 0.0, 'Frequency_changes': False}
 {'Played_frequency': 1727.0, 'Condition': -1.0, 'Block': 0.0, 'Frequency_changes': False}
 {'Played_frequency': 1727.0, 'Condition': -1.0, 'Block': 0.0, 'Frequency_changes': False}
 {'Played_frequency': 1727.0, 'Condition': -1.0, 'Block': 0.0, 'Frequency_changes': False}
 {'Played_frequency': 1727.0, 'Condition': -1.0, 'Block': 0.0, 'Frequency_changes': False}
 {'Played_frequency': 1727.0, 'Condition': -1.0, 'Block': 0.0, 'Frequency_changes': False}
 {'Played_frequency': 1727.0, 'Condition': -1.0, 'Block': 0.0, 'Frequency_changes': False}
 {'Played_frequency': 1727.0, 'Condition': -1.0, 'Block': 0.0, 'Frequency_changes': False}
 {'Played_frequency': 1727.0, 'Condition': -1.0, 'Block': 0.0, 'Frequency_changes': False}
 {'Played_frequency': 1727.0, 'Condition': -1.0, 'Block': 0.0, 'Frequency_changes': False}]


In [70]:
tone_vec = mapping.convert_to_frequency(x)
print(len(tone_vec))

527989


In [71]:
def freq(l):
    result = []
    for i in range(len(l)):
        # Ajoute l'élément si c'est le premier ou s'il est différent du dernier ajouté
        if not result or l[i] != result[-1]:
            result.append(l[i])
    return result

In [18]:
print(object)

<class 'object'>


In [88]:
print(len(freq(tone_vec)))#[12490:12510])
print(len(freq(tt['tones'])))#[12490:12510])

12510
11790


In [82]:
def ajuster(tone_vec,tt):
    pos = []
    freq_tt = []
    f_pos = tone_vec
    f_tt = tt['tones']
    n_pos = len(f_pos)
    n_tt = len(f_tt)
    i_pos = 0
    i_tt = 0
    poubelle =[]
    while i_tt<n_tt and i_pos<n_pos:
        if f_pos[i_pos] == f_tt[i_tt]:
            pos.append(f_pos[i_pos])
            freq_tt.append(f_tt[i_tt])
            i_pos += 1
            i_tt += 1
        else :
            poubelle.append([i_pos,f_pos[i_pos]])
            i_pos += 1
            

    return pos,freq_tt,poubelle

In [87]:
#print(ajuster(tone_vec,tt)[0] == ajuster(tone_vec,tt)[1])
#print(ajuster(tone_vec,tt)[2][0:100])
print(len(ajuster(tone_vec,tt)[0]))

2590


In [8]:
tracking,label_tracking = get_psth(data, features,t_pre, t_post, bin_width, gc, 'tracking',[530,720])
tracking_condense = get_part_psth(tracking, gc)
tracking_transpose = np.moveaxis(tracking_condense, 1, 0) #(4104, 33, 2)
label_tracking_transpose = np.moveaxis(label_tracking, 1, 0)

In [28]:
k= 0
while features[k]['Played_frequency'] == 0.0:
    k +=1
print(k)


1100


In [31]:
print([features[k]['Played_frequency'] for k in range(1100,1200)])
print(tone_vec[0:100])

[1490.0, 1490.0, 1490.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1490.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0, 1287.0]
[1490. 1490. 1287. 1287. 1287. 1287. 1287. 1490. 1287. 1287. 1287. 1287.
 1287. 1287. 1287. 1287. 1287. 1287. 1287. 1287. 1287. 1287. 1287. 1287.
 1287. 1287. 1287. 1287. 1287. 1287. 1287. 1287. 1287

In [26]:
print(len(tone_vec),len(clean_x_p))
print(len(features))

47998 47998
721980
